In [123]:
#pip install huggingface_hub
from langchain_community.llms import HuggingFaceHub

## Set your Hugging Face API token

In [125]:
huggingfacehub_api_token = 'TOKEN'

In [207]:
llm = HuggingFaceHub(repo_id='tiiuae/falcon-7b-instruct', huggingfacehub_api_token=huggingfacehub_api_token, model_kwargs = {
                "min_length": 200,
                "max_length": 1000,
                "temperature":0.5,
                "num_return_sequences":2,
                "max_new_tokens":512
            })

In [209]:
input = 'I want to open a restaurant for Indian food. Suggest a fancy name for this.'
output = llm.invoke(input)
print(output)

I want to open a restaurant for Indian food. Suggest a fancy name for this.
How about 'Taste of India Bistro' or 'Indigo Restaurant'?


In [55]:
llm('I want to open a restaurant for Indian food. Suggest a fancy name for this.')

"I want to open a restaurant for Indian food. Suggest a fancy name for this.\nHow about 'The Spice Kitchen' or 'Masala Maven'?"

In [57]:
llm('I want to open a restaurant for Thai food. Suggest a fancy name for this.')

"I want to open a restaurant for Thai food. Suggest a fancy name for this.\nHow about 'Siam Splendor' or 'Taste of Thailand'?"

## Prompt Templates

In [73]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
p = prompt_template_name.format(cuisine="Italian")
print(p)


I want to open a restaurant for Italian food. Suggest a fancy name for this.


## Chains

In [76]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.invoke("Mexican")

{'cuisine': 'Mexican',
 'text': 'I want to open a restaurant for Mexican food. Suggest a fancy name for this.\nLa Fonda del Sol Mexican Cuisine'}

In [79]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.invoke("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fancy name for this.

> Finished chain.


{'cuisine': 'Mexican',
 'text': 'I want to open a restaurant for Mexican food. Suggest a fancy name for this.\nLa Fonda del Sol Mexican Cuisine'}

In [89]:
llm = HuggingFaceHub(repo_id='tiiuae/falcon-7b-instruct', huggingfacehub_api_token=huggingfacehub_api_token, model_kwargs = {
                "min_length": 200,
                "max_length":2000,
                "temperature":0.6,
                "num_return_sequences":1
            })

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

#### Simple Sequential Chain

In [91]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.invoke("Indian")
print(content)

{'input': 'Indian', 'output': "Suggest some menu items for I want to open a restaurant for Indian food. Suggest a fancy name for this.\nIshaan Indian Cuisine or The Maharaja's Kitchen are both fancy names for an Indian restaurant. Some menu items to consider offering could be: Tandoori chicken, Palak paneer, Aloo Gobi, Biryani, and Samosas."}


#### Sequential Chain

In [127]:
llm = HuggingFaceHub(repo_id='tiiuae/falcon-7b-instruct', huggingfacehub_api_token=huggingfacehub_api_token, model_kwargs = {
                "min_length": 200,
                "max_length":2000,
                "temperature":0.6,
                "num_return_sequences":1
            })
prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")

In [220]:
llm = HuggingFaceHub(repo_id='tiiuae/falcon-7b-instruct', huggingfacehub_api_token=huggingfacehub_api_token, model_kwargs = {
                "min_length": 200,
                "max_length":2000,
                "temperature":0.6,
                "num_return_sequences":1
            })

prompt_template_items = PromptTemplate(
    input_variables = ['cuisine'],
    template="Suggest some menu items for {cuisine} restaurant."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [222]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [224]:
chain.invoke({"cuisine": "Indian"})

{'cuisine': 'Indian',
 'restaurant_name': "I want to open a restaurant for Indian food. Suggest a fancy name for this.\nIshaan Indian Cuisine or The Maharaja's Kitchen are both fancy names for an Indian restaurant.",
 'menu_items': 'Suggest some menu items for Indian restaurant.\n- Samosas\n- Tandoori chicken\n- Palak paneer\n- Aloo Gobi\n- Biryani\n- Tawa Gosht\n- Chana Masala'}